In [491]:
!unzip /content/songs.csv.zip

Archive:  /content/songs.csv.zip
replace songs.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [492]:
!unzip /content/echonest-metrics.json.zip

Archive:  /content/echonest-metrics.json.zip
replace echonest-metrics.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [493]:
import pandas as pd
import numpy  as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report



In [494]:
data = pd.read_csv("songs.csv")
echonest = pd.read_json("echonest-metrics.json")
echo_tracks = pd.merge(left=echonest, right=data[['track_id','bit_rate','duration','genre_top']], on='track_id')

In [495]:
echonest.head()

,track_id,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence
0,2,0.416675,0.675894,0.634476,0.010628,0.177647,0.159310,165.922,0.576661
1,3,0.374408,0.528643,0.817461,0.001851,0.105880,0.461818,126.957,0.269240
2,5,0.043567,0.745566,0.701470,0.000697,0.373143,0.124595,100.260,0.621661
3,10,0.951670,0.658179,0.924525,0.965427,0.115474,0.032985,111.562,0.963590
4,134,0.452217,0.513238,0.560410,0.019443,0.096567,0.525519,114.290,0.894072


In [518]:
echonest.isnull().any()

track_id            False
acousticness        False
danceability        False
energy              False
instrumentalness    False
liveness            False
speechiness         False
tempo               False
valence             False
dtype: bool

In [496]:
data.head()

,track_id,bit_rate,comments,composer,date_created,date_recorded,duration,favorites,genre_top,genres,...,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
0,135,256000,1,NaN,2008-11-26 01:43:26,2008-11-26 00:00:00,837,0,Rock,"[45, 58]",...,NaN,2484,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1832,NaN,0,NaN,[],Father's Day
1,136,256000,1,NaN,2008-11-26 01:43:35,2008-11-26 00:00:00,509,0,Rock,"[45, 58]",...,NaN,1948,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1498,NaN,0,NaN,[],Peel Back The Mountain Sky
2,151,192000,0,NaN,2008-11-26 01:44:55,NaN,192,0,Rock,[25],...,NaN,701,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,148,NaN,4,NaN,[],Untitled 04
3,152,192000,0,NaN,2008-11-26 01:44:58,NaN,193,0,Rock,[25],...,NaN,637,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,98,NaN,11,NaN,[],Untitled 11
4,153,256000,0,Arc and Sender,2008-11-26 01:45:00,2008-11-26 00:00:00,405,5,Rock,[26],...,NaN,354,en,Attribution-NonCommercial-NoDerivatives (aka M...,424,NaN,2,NaN,[],Hundred-Year Flood


In [497]:
echo_tracks.head()

,track_id,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence,bit_rate,duration,genre_top
0,2,0.416675,0.675894,0.634476,0.010628,0.177647,0.159310,165.922,0.576661,256000,168,Hip-Hop
1,3,0.374408,0.528643,0.817461,0.001851,0.105880,0.461818,126.957,0.269240,256000,237,Hip-Hop
2,5,0.043567,0.745566,0.701470,0.000697,0.373143,0.124595,100.260,0.621661,256000,206,Hip-Hop
3,134,0.452217,0.513238,0.560410,0.019443,0.096567,0.525519,114.290,0.894072,256000,207,Hip-Hop
4,153,0.988306,0.255661,0.979774,0.973006,0.121342,0.051740,90.241,0.034018,256000,405,Rock


In [519]:
echo_tracks.isnull().any()

acousticness        False
danceability        False
energy              False
instrumentalness    False
liveness            False
speechiness         False
tempo               False
valence             False
bit_rate            False
duration            False
genre_top           False
dtype: bool

In [498]:
data.describe()

,track_id,bit_rate,comments,duration,favorites,interest,listens,number
count,17734.000000,17734.000000,17734.000000,17734.000000,17734.000000,1.773400e+04,17734.000000,17734.000000
mean,69394.145991,258971.413781,0.020695,220.942934,1.685237,1.678238e+03,1079.709541,6.184110
std,45784.589753,61535.621463,0.270070,177.906821,6.629402,9.394969e+03,4693.497635,7.075062
min,2.000000,-1.000000,0.000000,2.000000,0.000000,1.800000e+01,9.000000,0.000000
25%,29979.000000,220982.000000,0.000000,139.000000,0.000000,4.210000e+02,214.000000,2.000000
50%,64193.500000,256000.000000,0.000000,192.000000,1.000000,8.240000e+02,461.000000,5.000000
75%,109442.750000,320000.000000,0.000000,255.000000,2.000000,1.628000e+03,1018.000000,8.000000
max,155319.000000,448000.000000,24.000000,3675.000000,554.000000,1.038669e+06,433992.000000,100.000000


In [499]:
echo_tracks=echo_tracks.drop(columns=['track_id'])

In [520]:
col=len(echo_tracks.axes[1])
print("count of columns= ",col)

count of columns=  11


In [501]:
x=np.array(echo_tracks.iloc[:,0:col-1])
y=np.array(echo_tracks.iloc[:,col-1])

In [502]:
#encode labels to transform from rock and hiphop to 0,1
le = preprocessing.LabelEncoder()
le.fit(y)
y=le.transform(y)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

In [503]:
#normailze features to make all feature same weight in updating weights
X_train_stand = x_train.copy()
X_test_stand = x_test.copy()
scale = StandardScaler().fit(X_train_stand)
X_train_stand = scale.transform(X_train_stand)
X_test_stand = scale.transform(X_test_stand)


In [504]:
clf=SVC(C=.7,kernel="rbf",coef0=2,degree=2,gamma=0.1,class_weight={0:0.81,1:0.19})
clf.fit(X_train_stand, y_train)

SVC(C=0.7, class_weight={0: 0.81, 1: 0.19}, coef0=2, degree=2, gamma=0.1)

In [505]:
y_predict=clf.predict(X_test_stand)
accuracy_score(y_test, y_predict)
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.71      0.88      0.79       225
           1       0.97      0.92      0.94       976

    accuracy                           0.91      1201
   macro avg       0.84      0.90      0.87      1201
weighted avg       0.92      0.91      0.92      1201



accuray of model well but precision of class 0 not enough


---


Do when make the data balanced well change accuracy

In [506]:
Rock=np.array(echo_tracks[echo_tracks.iloc[:,col-1]=="Rock"])
Hip=np.array(echo_tracks[echo_tracks.iloc[:,col-1]=="Hip-Hop"])
index = np.random.choice(Rock.shape[0], Hip.shape[0], replace=False)
Rock=Rock[index]
newData=np.concatenate((Rock, Hip), axis=0)
np.random.shuffle(newData)
np.random.shuffle(newData)
np.random.shuffle(newData)

In [511]:
#encode labels to transform from rock and hiphop to 0,1
x=np.array(newData[:,0:col-1])
y=np.array(newData[:,col-1])
le = preprocessing.LabelEncoder()
le.fit(y)
y=le.transform(y)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

In [512]:
#normailze features to make all feature same weight in updating weights
X_train_stand = x_train.copy()
X_test_stand = x_test.copy()
scale = StandardScaler().fit(X_train_stand)
X_train_stand = scale.transform(X_train_stand)
X_test_stand = scale.transform(X_test_stand)

In [513]:
clf=SVC(C=8,kernel="rbf",coef0=1,degree=2,gamma=.01)
clf.fit(X_train_stand, y_train)

SVC(C=8, coef0=1, degree=2, gamma=0.01)

In [514]:
y_predict=clf.predict(X_test_stand)
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.91      0.86      0.89       211
           1       0.89      0.93      0.91       244

    accuracy                           0.90       455
   macro avg       0.90      0.89      0.90       455
weighted avg       0.90      0.90      0.90       455



balanced data improve the precision of class 0